In [1]:
!pip install polars
!pip install sentence-transformers
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 KB 7.3 MB/s eta 0:00:00a 0:00:01
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import polars as pl
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import datetime
import re

/home/tiao553/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-22 22:11:44.031626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753233104.122068  257125 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753233104.147757  257125 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753233104.324207  257125 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 0

In [3]:

resultado_final_df_polars = pl.read_parquet("../data/feature_store/resultado_final.parquet")
resultado_final_df_polars.head(10).to_pandas()


,codigo_vaga,titulo,modalidade,codigo_candidato,p_situacao_candidado,p_data_candidatura,p_ultima_atualizacao,p_comentario,p_recrutador,app_ib_objetivo_profissional,...,job_area_formacao,app_principais_ferramentas_tecnologicas,app_ferramentas_tecnologicas,app_competencias_tecnicas,app_competencias_comportamentais,app_experiencia_anos,app_senioridade_aparente,app_formacao_academica,app_nivel_formacao,app_area_formacao
0,4530,CONSULTOR CONTROL M,,25632,ENCAMINHADO AO REQUISITANTE,2021-03-25,2021-03-25,"ENCAMINHADO PARA - PJ R 72,00/HORA",ANA LÍVIA MOREIRA,,...,TI,"[PL/SQL, ITIL, Java, DB2, Python, SQL, HTML, D...","[Control-M, PowerCenter, IBM Cloud, Docker, Ku...","[PL/SQL, ITIL, SCM, DB2, Java, Python, SQL, Da...","[Comunicação, Trabalho em equipe, Liderança, R...",8-10 anos,Sênior,Superior Completo,Superior,Tecnologia da Informação
1,4530,CONSULTOR CONTROL M,,25529,ENCAMINHADO AO REQUISITANTE,2021-03-22,2021-03-23,"ENCAMINHADO PARA - R 6.000,00 CLT FULL , NÃO ...",ANA LÍVIA MOREIRA,ANALISTA DE REDES E TELEPROCESSAMENTO/INFRAEST...,...,TI,"[Redes de computadores, Virtualização VMware, ...","[Redes LAN/WAN, Virtualização VMware, Ferramen...","[Administração de Data Centers, Gerenciamento ...","[Comunicação, Trabalho em Equipe, Liderança, A...",8,Pieno,Superior Completo,Superior Completo,TI
2,4531,2021-2607395-PEOPLESOFT APPLICATION ENGINE-DOM...,,25364,CONTRATADO PELA DECISION,2021-03-17,2021-04-12,DATA DE INICIO: 12/04/2021,JULIANA CASSIANO,LIDERANÇA / DESENVOLVIMENTO,...,TI,"[SAP, Java, Python, SQL, Oracle, Power BI, Clo...","[SAP ERP, Java, SQL, Oracle, Power BI, Cloud P...","[SAP Implementation, Java Development, Cloud A...","[Problem Solving, Teamwork, Communication, Lea...",8-10 anos,Sênior,Superior Completo,Superior Completo,TI
3,4531,2021-2607395-PEOPLESOFT APPLICATION ENGINE-DOM...,,25360,ENCAMINHADO AO REQUISITANTE,2021-03-17,2021-03-17,,JULIANA CASSIANO,,...,TI,"[Oracle PeopleSoft, SAP, ABAP, SQL, Relatórios...","[Oracle PeopleTools, SAP, ABAP, SQL, Crystal, ...","[Desenvolvimento, Implementação, Configuração,...","[Resolução de problemas, Comunicação, Trabalho...",8-10 anos,Sênior,true,Superior Completo,TI
4,4533,2021-2605708-MICROFOCUS APPLICATION LIFE CYCLE...,,26338,CONTRATADO PELA DECISION,2021-04-29,2021-05-18,,STELLA VIEIRA,,...,TI,"[Testes, Automação, Integração, Performance, B...","[Java, C#, SQL, Oracle, MySQL, SharePoint, SAP...","[Testes, Automação, Integração, Performance, B...","[Proatividade, Liderança, Adaptabilidade, Comu...",5-8 anos,Pleno,true,Superior Completo,TI
5,4533,2021-2605708-MICROFOCUS APPLICATION LIFE CYCLE...,,24645,DESISTIU,2021-04-27,2021-04-27,PROFISSIONAL DESISTIU DA VAGA. MOTIVO : NA VER...,YASMIN DA ROSA,,...,TI,"[Selenium, WebDriver, Swagger, Gherkin]","[Java, Webdriver, Selenium, Swagger, BDD, Gher...","[Testes Funcionais, Testes Automatizados, Test...","[Aprendizado Contínuo, Proativo, Curiosidade, ...",0-2 anos,Júnior,true,Superior Completo,TI
6,4534,2021-2605711-MICROFOCUS QTP - UFT AUTOMATION T...,,26361,DOCUMENTAÇÃO PJ,2021-04-28,2021-05-11,AGUARDANDO CONFIRMAÇÃO DE INICIO _,MANUELLA CARVALHO,,...,TI,"[Selenium, Java, Eclipse, SQL, Azure DevOps, W...","[C, C#, WPF, Java, Selenio, Eclipse, SQL, Azur...","[Desenvolvimento, Testes, Automação, Regressão...","[Análise, Resolução, Implementação, Liderança,...",5-8 anos,Pleno,true,Superior Completo,Engenharia de Software
7,4534,2021-2605711-MICROFOCUS QTP - UFT AUTOMATION T...,,26205,DESISTIU,2021-04-23,2021-04-30,"PROFISSIONAL NÃO ATENDE OU RESPONDE LIGAÇÕES, ...",ANA CAMARGO,,...,TI,"[Selenium, Java, JavaScript, BDD, API, Testes,...","[Selenium Web Driver, WebDriver IO, Chai, Moch...","[Automação de testes, Desenvolvimento BDD, Tes...","[Proatividade, Análise Crítica, Resolução de P...",5-8 anos,Pleno,true,Superior Completo,TI
8,4534,2021-2605711-MICROFOCUS QTP - UFT AUTOMATION T...,,26003,NÃO APROVADO PELO CLIENTE,2021-04-08,2021-04-16,"CONVERSANDO COM A CANDIDATA, FOI EXPOSTO QUE A...",CAROLINA ARAÚJO,,...,TI,"[Selenium, Python,

In [4]:
# Combina as listas de skills (igual ao código anterior)
skills_extracted_df = resultado_final_df_polars.with_columns(
    (pl.col("job_ferramentas_tecnologicas").list.concat(pl.col("job_competencias_tecnicas"))).list.unique().alias("job_total_skills_list"),
    (pl.col("app_ferramentas_tecnologicas").list.concat(pl.col("app_competencias_tecnicas"))).list.unique().alias("app_total_skills_list")
)


In [5]:
def calculate_soft_match_score(job_skills, app_skills, model, threshold=0.5):
    """
    Calcula um score de similaridade semântica entre duas listas de skills.
    """
    # Retorna 0 se alguma das listas estiver vazia
    if not job_skills or not app_skills:
        return 0.0

    # Converte cada skill em um vetor numérico
    job_embeddings = model.encode(job_skills)
    app_embeddings = model.encode(app_skills)

    # Calcula a matriz de similaridade de cosseno entre todas as skills
    similarity_matrix = cosine_similarity(job_embeddings, app_embeddings)

    # Para cada skill da vaga, encontra a maior similaridade na lista do candidato
    best_matches = np.max(similarity_matrix, axis=1)

    # Aplica um threshold: se a melhor correspondência for muito baixa, consideramos 0
    # Isso evita que "Excel" dê match com "Java", por exemplo.
    qualified_matches = [score if score >= threshold else 0 for score in best_matches]
    
    # O score final
    final_score = np.mean(qualified_matches)
    
    return final_score

In [6]:
# Carrega o modelo de IA. 
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [7]:

skills_extracted_df = skills_extracted_df.with_columns(
    pl.struct(["job_total_skills_list", "app_total_skills_list"])
    .map_elements(
        lambda cols: calculate_soft_match_score(
            cols["job_total_skills_list"],
            cols["app_total_skills_list"],
            model
        ),
        return_dtype=pl.Float64
    ).alias("soft_match_score")
)

### FASE2

In [8]:

level_map = {
    # Idiomas
    'NENHUM': 0, 'BÁSICO': 1, 'INTERMEDIÁRIO': 2, 'AVANÇADO': 3, 'FLUENTE': 4, 'TÉCNICO': 1.5,
    
    # Níveis Profissionais
    'JÚNIOR': 1, 'JUNIOR': 1,
    'PLENO': 2,
    'SÊNIOR': 3, 'SENIOR': 3,
    'ESPECIALISTA': 4,
    'LÍDER': 5,
    
    # Níveis Acadêmicos
    'ENSINO MÉDIO COMPLETO': 1, 
    'ENSINO SUPERIOR INCOMPLETO': 2, 
    'ENSINO SUPERIOR COMPLETO': 3, 
    'PÓS-GRADUAÇÃO': 4,
    'MESTRADO': 4,
    'DOUTORADO': 4
}

#FAZ UM MAP DAS chaves do dicionário para upper e strip
def get_level(level_str):
    return level_map.get(str(level_str).upper().strip(), 0)

#
def calculate_structured_score(job_level, app_level):
    """
    Calcula uma pontuação proporcional baseada nos níveis.
    """
    job_score = get_level(job_level)
    app_score = get_level(app_level)

    # Se a vaga não exige nível, o candidato atende 100%
    if job_score == 0:
        return 1.0
    
    # Se o candidato não tem nível, a pontuação é 0
    if app_score == 0:
        return 0.0

    # Calcula a proporção e limita o score máximo a 1.0 (100%)
    ratio = app_score / job_score
    return min(ratio, 1.0)




In [9]:
skills_extracted_df = skills_extracted_df.with_columns(
    professional_level_score=pl.struct(["job_pv_nivel_profissional", "app_senioridade_aparente"]).map_elements(lambda x: calculate_structured_score(x['job_pv_nivel_profissional'], x['app_senioridade_aparente']), return_dtype=pl.Float64),
    academic_level_score=pl.struct(["job_pv_nivel_academico", "app_formacao_academica"]).map_elements(lambda x: calculate_structured_score(x['job_pv_nivel_academico'], x['app_formacao_academica']), return_dtype=pl.Float64),
    english_level_score=pl.struct(["job_pv_nivel_ingles", "app_fei_nivel_ingles"]).map_elements(lambda x: calculate_structured_score(x['job_pv_nivel_ingles'], x['app_fei_nivel_ingles']), return_dtype=pl.Float64)
)

In [10]:
structured_weights = {'professional': 0.6, 'academic': 0.2, 'english': 0.2}

skills_extracted_df = skills_extracted_df.with_columns(
    ((pl.col("professional_level_score") * structured_weights['professional']) +
     (pl.col("academic_level_score") * structured_weights['academic']) +
     (pl.col("english_level_score") * structured_weights['english']))
    .alias("structured_match_score")
)


### FASE 3

In [11]:
job_text = skills_extracted_df["job_pv_principais_atividades"].fill_null("").to_list()
app_text = skills_extracted_df["app_cv_pt"].fill_null("").to_list()

job_embeddings = model.encode(job_text, show_progress_bar=False)
app_embeddings = model.encode(app_text, show_progress_bar=False)

cosine_scores = [cosine_similarity([job_embeddings[i]], [app_embeddings[i]])[0][0] for i in range(len(skills_extracted_df))]
skills_extracted_df = skills_extracted_df.with_columns(pl.Series("cosine_similarity_score", cosine_scores))

In [ ]:
# --- 0. IMPORTAÇÃO DAS BIBLIOTECAS ---
import polars as pl
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --- 1. FUNÇÕES DE APOIO E DICIONÁRIOS (O "CÉREBRO" DO MODELO) ---

# Dicionário de Níveis, incluindo as suas melhorias
level_map = {
    # Idiomas
    'NENHUM': 0, 'BÁSICO': 1, 'TÉCNICO': 1.5, 'INTERMEDIÁRIO': 2, 'AVANÇADO': 3, 'FLUENTE': 4,
    
    # Níveis Profissionais
    'JÚNIOR': 1, 'JUNIOR': 1,
    'PLENO': 2,
    'SÊNIOR': 3, 'SENIOR': 3,
    'ESPECIALISTA': 4,
    'LÍDER': 5,
    
    # Níveis Acadêmicos
    'ENSINO MÉDIO COMPLETO': 1,
    'ENSINO SUPERIOR INCOMPLETO': 2,
    'ENSINO SUPERIOR COMPLETO': 3,
    'PÓS-GRADUAÇÃO': 4,
    'MESTRADO': 4,   # Sua adição
    'DOUTORADO': 4  # Sua adição
}

def get_level(level_str: str) -> float:
    """Função segura para traduzir um nível em texto para seu valor numérico."""
    return level_map.get(str(level_str).upper().strip(), 0)

def calculate_structured_score(job_level: str, app_level: str) -> float:
    """Calcula uma pontuação proporcional para os requisitos estruturados."""
    job_score = get_level(job_level)
    app_score = get_level(app_level)
    
    if job_score == 0:
        return 1.0
    if app_score == 0:
        return 0.0
        
    ratio = app_score / job_score
    return min(ratio, 1.0)

def calculate_soft_match_score(job_skills: list, app_skills: list, model, threshold=0.5) -> float:
    """Calcula o score de similaridade semântica entre listas de skills."""
    if not job_skills or not app_skills:
        return 0.0

    job_embeddings = model.encode(job_skills)
    app_embeddings = model.encode(app_skills)
    similarity_matrix = cosine_similarity(job_embeddings, app_embeddings)
    
    best_matches = np.max(similarity_matrix, axis=1)
    qualified_matches = [score if score >= threshold else 0 for score in best_matches]
    
    return np.mean(qualified_matches)

# --- 2. CARREGAMENTO DOS DADOS E DO MODELO DE IA ---

print("Carregando modelo de IA (SentenceTransformer)... Isso pode levar um momento.")
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print("Modelo de IA carregado com sucesso.")

# Carregue seu DataFrame completo aqui.
# Em um cenário real: df = pl.read_json('seu_arquivo_de_1_milhao_de_linhas.json')
# Para demonstração, usamos uma amostra dos seus dados:
data = [
    {'codigo_vaga': '4530', 'codigo_candidato': '25632', 'job_pv_nivel_profissional': 'PLENO', 'job_pv_nivel_academico': 'ENSINO SUPERIOR COMPLETO', 'job_pv_nivel_ingles': 'AVANÇADO', 'job_pv_principais_atividades': 'Experiência em projetos de Control-M e automação de processos batch.', 'app_senioridade_aparente': 'Sênior', 'app_formacao_academica': 'Pós-graduação', 'app_fei_nivel_ingles': 'AVANÇADO', 'app_cv_pt': 'Sólida experiência com a ferramenta de automação Control-M, atuando na criação de jobs em Shell Script e otimização com PL/SQL. Conhecimentos em Python para scripts de apoio e vivência com bancos de dados Oracle.',
     'job_ferramentas_tecnologicas': ['Control-M', 'SQL', 'Scripting'], 'job_competencias_tecnicas': ['Análise', 'Desenvolvimento', 'Projetos'], 'app_ferramentas_tecnologicas': ['Control-M', 'PostgreSQL', 'Docker', 'Python'], 'app_competencias_tecnicas': ['PL/SQL', 'Análise de Dados', 'Liderança', 'Shell Script']}
]
df = pl.DataFrame(data)

# --- 3. EXECUÇÃO DO PIPELINE DE ANÁLISE ---

print("\nIniciando o processamento do pipeline de ranking...")

# Combina as listas de skills da vaga e do candidato
df = df.with_columns(
    (pl.col("job_ferramentas_tecnologicas").list.concat(pl.col("job_competencias_tecnicas"))).list.unique().alias("job_total_skills_list"),
    (pl.col("app_ferramentas_tecnologicas").list.concat(pl.col("app_competencias_tecnicas"))).list.unique().alias("app_total_skills_list")
)

# --- ETAPA 1: "Soft Matching" com Embeddings de Skills ---
print("Etapa 1: Calculando 'Soft Match' das skills...")
df = df.with_columns(
    pl.struct(["job_total_skills_list", "app_total_skills_list"])
    .map_elements(
        lambda cols: calculate_soft_match_score(
            cols["job_total_skills_list"],
            cols["app_total_skills_list"],
            model
        ),
        return_dtype=pl.Float64
    ).alias("soft_match_score")
)

# --- ETAPA 2: Matching de Dados Estruturados (com Média Ponderada) ---
print("Etapa 2: Calculando score de requisitos estruturados...")
df = df.with_columns(
    professional_level_score=pl.struct(["job_pv_nivel_profissional", "app_senioridade_aparente"]).map_elements(lambda x: calculate_structured_score(x['job_pv_nivel_profissional'], x['app_senioridade_aparente']), return_dtype=pl.Float64),
    academic_level_score=pl.struct(["job_pv_nivel_academico", "app_formacao_academica"]).map_elements(lambda x: calculate_structured_score(x['job_pv_nivel_academico'], x['app_formacao_academica']), return_dtype=pl.Float64),
    english_level_score=pl.struct(["job_pv_nivel_ingles", "app_fei_nivel_ingles"]).map_elements(lambda x: calculate_structured_score(x['job_pv_nivel_ingles'], x['app_fei_nivel_ingles']), return_dtype=pl.Float64)
)
structured_weights = {'professional': 0.6, 'academic': 0.2, 'english': 0.2}
df = df.with_columns(
    ((pl.col("professional_level_score") * structured_weights['professional']) +
     (pl.col("academic_level_score") * structured_weights['academic']) +
     (pl.col("english_level_score") * structured_weights['english']))
    .alias("structured_match_score")
)

# --- ETAPA 3: Similaridade de Contexto do CV (Cosine Similarity) ---
print("Etapa 3: Calculando similaridade de contexto do CV...")
job_text = df["job_pv_principais_atividades"].fill_null("").to_list()
app_text = df["app_cv_pt"].fill_null("").to_list()
job_embeddings = model.encode(job_text, show_progress_bar=False)
app_embeddings = model.encode(app_text, show_progress_bar=False)
cosine_scores = [cosine_similarity([job_embeddings[i]], [app_embeddings[i]])[0][0] for i in range(len(df))]
df = df.with_columns(pl.Series("cosine_similarity_score", cosine_scores))

# --- 4. ETAPA FINAL: CÁLCULO DO `technical_score` ---
print("Etapa Final: Calculando a pontuação final ponderada...")
final_weights = {
    'soft_match': 0.60,
    'cosine': 0.25,
    'structured': 0.15
}
df = df.with_columns(
    (pl.col("soft_match_score") * final_weights['soft_match'] +
     pl.col("structured_match_score") * final_weights['structured'] +
     pl.col("cosine_similarity_score") * final_weights['cosine'])
    .alias("technical_score")
)

# --- 5. RESULTADO FINAL ---
final_df = df.select([
    "codigo_vaga",
    "codigo_candidato",
    pl.col("soft_match_score").round(3).alias("Score de Skills"),
    pl.col("structured_match_score").round(3).alias("Score Estruturado"),
    pl.col("cosine_similarity_score").round(3).alias("Score de Contexto"),
    pl.col("technical_score").round(3).alias("PONTUAÇÃO FINAL")
]).sort("PONTUAÇÃO FINAL", descending=True)

print("\n--- RESULTADO FINAL DO MODELO DE RANKING ---")
print(final_df)